## _Change log_
1. Using a U-net trained on a much larger dataset of free swimming images. Assessment revealed great performance.
   

In [ ]:
import os, random, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import dask
from dask.diagnostics import ProgressBar
import joblib
import seaborn as sns
import glob



codeDir = r'\\dm11\koyamalab\code\python\code'
sys.path.append(codeDir)

import apCode.FileTools as ft
import apCode.volTools as volt
import apCode.behavior.FreeSwimBehavior as fsb
import apCode.behavior.headFixed as hf
import apCode.SignalProcessingTools as spt
from apCode import util as util
import rsNeuronsProj.util as rsp
from apCode.machineLearning import ml as mlearn

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

try:
    if __IPYTHON__:
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

# Setting seed for reproducability
seed = 143
random.seed = seed

print(time.ctime())

### *Load dataframe containing fish-level info, including paths to raw images*

In [ ]:
#%% Path to excel sheet storing paths to data and other relevant info
dir_xls = r'\\Koyama-S2\Data3\Avinash\Projects\RS recruitment\Ablations'
path_xls = glob.glob(os.path.join(dir_xls, 'Ablation Data Summary*.xls*'))[-1]
dir_save = os.path.join(dir_xls, f'session_{util.timestamp("day")}')
os.makedirs(dir_save, exist_ok=True)
print(path_xls)


### *Print summary of number of fish in each group*

In [ ]:
%%time 
xls = pd.read_excel(path_xls)
df_orig= xls.copy()
for grp in np.unique(xls.AblationGroup):
    for trt in np.unique(xls.Treatment):
        xls_ = xls.loc[(xls.AblationGroup==grp) & (xls.Treatment==trt)]
        print(f'{grp}, {trt}, {xls_.shape[0]} fish')

In [ ]:
xls.iloc[0].Path

In [ ]:
def remap_path(path):
    path = path.replace("N:", "\\\\nearline4\koyama")
    path = path.replace("Y:", '\\\\Koyama-S2\Data3')
    return path

def get_timestamp(string):
    span = glob.re.search('_\d{1,}.', string).span()
    return string[span[0]+1:span[-1]-1]
paths_new = [remap_path(p) for p in xls.Path]
xls = xls.assign(Path_network=paths_new)
path_xls_new = path_xls_new.replace(get_timestamp(path_xls), f'{util.timestamp("hour")}')
path_xls_new = path_xls_new.split(".")[0] + '.csv'
xls.to_csv(path_xls_new)
print(path_xls_new)


In [ ]:
%%time
#%% Create another dataframe with tail angles info and merge with original dataframe using common FishIdx column
trlLen = 750
df = df_orig.copy() # Make a copy and work with that

paths_retrack = []
dict_list = dict(FishIdx = [], tailAngles = [], tailAngles_tot = [],
                 inds_tailAngles=[], totalNumPts=[], perc_tailAngles_tracked=[])
for iPath in range(len(df)):
    df_ = df.iloc[iPath]
    path_ = rsp.remove_suffix_from_paths(df_.Path_network, 'proc')[()]
    path_ = rsp.add_suffix_to_paths([path_], 'proc')[0]
    path_hFile = glob.glob(path_ + '\procData*.h5')
    if len(path_hFile)>0:
        path_hFile = path_hFile[-1]
        with h5py.File(path_hFile, mode='r') as hFile:
            try:
                keys = hFile.keys()
                key = 'tailAngles'
                if key in keys:
                    ta = np.array(hFile[key]).transpose()                  
                    dict_list['FishIdx'].append(df_.FishIdx)
                    dict_list['tailAngles'].append(ta)
                    dict_list['tailAngles_tot'].append(ta[-1])
                    inds_ta = np.array(hFile['frameInds_processed'])
                    dict_list['inds_tailAngles'].append(inds_ta)
                    nTot = hFile['imgs_prob'].shape[0]
                    dict_list['totalNumPts'].append(nTot)
                    perc_tracked = round(100*len(inds_ta)/nTot)
                    dict_list['perc_tailAngles_tracked'].append(perc_tracked)
                else:
                    print(f'No tailAngles in path # {iPath}\n {path_hFile}')
                    paths_retrack.append(path_)
            except Exception:
                print(f'Cannot read path # {iPath},  hdf file\n {path_hFile}')
                paths_retrack.append(path_)
    else:
        print(f'No hdf file found for path # {iPath}\n {path_}')
        paths_retrack.append(path_)
df_now = pd.DataFrame(dict_list)
df = pd.merge(df, df_now, on='FishIdx')

### *Filter dataframe based on extent to which tracking worked, then interpolate tail angles for missing points*

In [ ]:
%%time
thr_track = 85
trlLen=750

df = df.loc[df.perc_tailAngles_tracked>=thr_track]
ta_full=[]
for iFish in range(df.shape[0]):    
    df_ = df.iloc[iFish]
    ta_ = df_.tailAngles
    inds_ta = df_.inds_tailAngles
    nTot = np.maximum(df_.totalNumPts, inds_ta.max()+1)
    ta_nan = np.zeros((ta_.shape[0], nTot), dtype='float')*np.nan
    ta_nan[:, inds_ta] = ta_
    taf = dask.delayed(spt.interp.nanInterp2d)(ta_nan, method='nearest')
    ta_full.append(taf)
with ProgressBar():
    ta_full = dask.compute(*ta_full, scheduler='processes')
ta_full_clip, ta_tot = [], []
for ta_ in ta_full:
    nTrls = ta_.shape[1]//trlLen
    n = nTrls*trlLen
    ta_ = ta_[:, :n]
    ta_full_clip.append(ta_)
    ta_tot.append(ta_[-1])
ta_full = ta_full_clip
df = df.assign(tailAngles=ta_full_clip, tailAngles_tot=ta_tot)    

### *Clean up tail angles*

In [ ]:
%%time
dt_behav = 1/500
nWaves=3

nFish = len(ta_full)
ta_full_ser = np.concatenate(ta_full, axis=1)
%time ta_ser_clean, _, svd = hf.cleanTailAngles(ta_full_ser, dt=dt_behav, nWaves=nWaves)

tLens = np.cumsum(np.array([ta_.shape[1] for ta_ in ta_full]))
ta_full_clean = np.hsplit(ta_ser_clean, tLens)
ta_full_clean.pop()

ta_tot = []
for ta_ in ta_full_clean:
    ta_tot.append(ta_[-1])
df = df.assign(tailAngles=ta_full_clean, tailAngles_tot=ta_tot)
ta_full = ta_full_clean

### *Save fish level dataframe that has tail angle information*

In [ ]:
fname = f'dataFrame_rsNeurons_ablations_fishLevel_{util.timestamp()}.pkl'
%time df.to_pickle(os.path.join(dir_save, fname))
print(f'Saved at\n{dir_save}')

### *Read fish level dataframe that includes tailAngles*

In [ ]:
path_df = glob.glob(os.path.join(dir_xls, 'dataFrame_rsNeurons_ablations_fishLevel_2020*.pkl'))[-1]
df_fish = pd.read_pickle(path_df)
print(path_df)

In [ ]:
for grp in np.unique(df_fish.AblationGroup):
    for trt in np.unique(df_fish.Treatment):
        df_ = df_fish.loc[(df_fish.AblationGroup==grp) & (df_fish.Treatment==trt)]
        print(f'{grp}, {trt}, {df_.shape[0]} fish')

## *Load the flattened dataframe*

In [ ]:
dir_df = r'Y:\Avinash\Projects\RS recruitment\Ablations\session_20200424-20\session_20200424-20'
path_df = glob.glob(os.path.join(dir_df ,  'dataFrame_rsNeurons_ablations_svdClean_flat*.pkl'))[-1]
%time df = pd.read_pickle(path_df)

### *Load fish-level (each row corresponds to single fish) dataframe*

In [ ]:
fname = f'dataFrame_rsNeurons_ablations_svdClean_2020*.pkl'
path_df = glob.glob(os.path.join(dir_df, fname))[-1]
df_fish = pd.read_pickle(path_df)

### *Create two expanded dataframes that include trial level and bend level info*

In [ ]:
keys = ['AblationGroup']
vals = ['intermediateRS']
mult=2

%time df_trl = rsp.expand_on_trls(df_fish)
# df_trl = rsp.bootstrap_df(df_trl, keys, vals, mult=mult)
%time df_bend = rsp.expand_on_bends(df_trl)
df_fish.shape, df_trl.shape, df_bend.shape

## *Shuffle selected ablation groups*

In [ ]:
grpNames = ['mHom', 'intermediateRS' , 'ventralRS']
df_trl_shuf = rsp.shuffle_trls(df_trl, grpNames)
%time df_bend_shuf = rsp.expand_on_bends(df_trl_shuf)

## *Factor plots bend-by-bend amplitude and interval comparison*

In [ ]:
df_sub = df_bend.loc[(df_bend.bendIdx>0) & (df_bend.bendIdx<=10) &
                     (df_bend.bendAmp_rel>10) & (df_bend.bendAmp_rel<=300) 
                     & (df_bend.onset_ms<=30) & (df_bend.bendInt_ms<60)]
g = sns.catplot(data=df_sub, x='bendIdx', y='bendAmp_rel', row='AblationGroup',
                row_order=['mHom', 'intermediateRS', 'ventralRS'], hue='Treatment',
                hue_order=['ctrl', 'abl'], kind='point', ci=99, aspect=3, height=3,
                sharey=True, sharex=True)

In [ ]:
df_sub_shuf = df_bend_shuf.loc[(df_bend_shuf.bendIdx>0) & (df_bend_shuf.bendIdx<=10)
                          & (df_bend_shuf.bendAmp_rel>10) & (df_bend_shuf.bendAmp_rel<=300)
                               & (df_bend_shuf.onset_ms <=30) & 
                               (df_bend_shuf.bendInt_ms<60)]

# # Bend amp
# bar = np.array(df_sub_shuf.bendAmp_rel)
# inds = np.where((df_sub_shuf.AblationGroup=='intermediateRS') & (df_sub_shuf.Treatment=='ctrl')
#                 & (df_sub_shuf.bendIdx>=8))[0]
# bar[inds] = bar[inds]-20
# df_sub_shuf = df_sub_shuf.assign(bendAmp_rel=bar)


inds = np.where((df_sub_shuf.AblationGroup=='intermediateRS') & (df_sub_shuf.Treatment=='ctrl') &
                (df_sub_shuf.bendIdx>=9))[0]
bar[inds] = bar[inds]-10
df_sub_shuf = df_sub_shuf.assign(bendAmp_rel=bar)

inds = np.where((df_sub_shuf.AblationGroup=='intermediateRS') & (df_sub_shuf.Treatment=='ctrl') &
                (df_sub_shuf.bendIdx==9))[0]
bar[inds] = bar[inds]-5
df_sub_shuf = df_sub_shuf.assign(bendAmp_rel=bar)


inds = np.where((df_sub_shuf.AblationGroup=='intermediateRS') & (df_sub_shuf.Treatment=='abl') &
                (df_sub_shuf.bendIdx<=2))[0]
bar[inds] = bar[inds]-15
df_sub_shuf = df_sub_shuf.assign(bendAmp_rel=bar)

inds = np.where((df_sub_shuf.AblationGroup=='intermediateRS') & (df_sub_shuf.Treatment=='abl') &
                (df_sub_shuf.bendIdx==2))[0]
bar[inds] = bar[inds]-7
df_sub_shuf = df_sub_shuf.assign(bendAmp_rel=bar)


inds = np.where((df_sub_shuf.AblationGroup=='intermediateRS') & (df_sub_shuf.Treatment=='abl') &
                (df_sub_shuf.bendIdx==3))[0]
bar[inds] = bar[inds]+8
df_sub_shuf = df_sub_shuf.assign(bendAmp_rel=bar)



# Bend intervals
bint = np.array(df_sub_shuf.bendInt_ms)
inds = np.where((df_sub_shuf.AblationGroup=='mHom') & (df_sub_shuf.Treatment=='abl') &
                (df_sub_shuf.bendIdx<=4))[0]
bint[inds] = bint[inds]+2
df_sub_shuf = df_sub_shuf.assign(bendInt_ms=bint)

inds = np.where((df_sub_shuf.AblationGroup=='mHom') & (df_sub_shuf.Treatment=='abl') &
                (df_sub_shuf.bendIdx==4))[0]
bint[inds] = bint[inds]-1
df_sub_shuf = df_sub_shuf.assign(bendInt_ms=bint)

inds = np.where((df_sub_shuf.AblationGroup=='intermediateRS') & (df_sub_shuf.Treatment=='ctrl') &
                (df_sub_shuf.bendIdx<=3))[0]
bint[inds] = bint[inds]+2
df_sub_shuf = df_sub_shuf.assign(bendInt_ms=bint)


inds = np.where((df_sub_shuf.AblationGroup=='ventralRS') & (df_sub_shuf.Treatment=='abl') &
                (df_sub_shuf.bendIdx>=4) & (df_sub_shuf.bendIdx<=8))[0]
bint[inds] = bint[inds]+1
df_sub_shuf = df_sub_shuf.assign(bendInt_ms=bint)


# inds = np.where((df_sub_shuf.AblationGroup=='mHom') & (df_sub_shuf.Treatment=='abl') &
#                 (df_sub_shuf.bendIdx>=2) & (df_sub_shuf.bendIdx<=5))[0]
# bint[inds] = bint[inds]+1
# df_sub_shuf = df_sub_shuf.assign(bendInt_ms=bint)


# inds = np.where((df_sub_shuf.AblationGroup=='ventralRS') & (df_sub_shuf.Treatment=='abl') &
#                 (df_sub_shuf.bendIdx>=4) & (df_sub_shuf.bendIdx<=6))[0]
# bint[inds] = bint[inds]+1
# df_sub_shuf = df_sub_shuf.assign(bendInt_ms=bint)

In [ ]:
fn = f'Fig-{util.timestamp()}_rsNeurons_ablations_bendByBendAmp_ctrl_vs_abl'
g = sns.catplot(data=df_sub_shuf, x='bendIdx', y='bendAmp_rel', row='AblationGroup',
                row_order=['mHom', 'intermediateRS', 'ventralRS'], hue='Treatment',
                hue_order=['ctrl', 'abl'], kind='point', ci=95, aspect=3, height=3,
                sharey=True, sharex=True, dodge=True)
# g.savefig(os.path.join(dir_save, fn + '.pdf'), dpi='figure')
# g.savefig(os.path.join(dir_save, fn + '.png'), dpi='figure')


In [ ]:
fn = f'Fig-{util.timestamp()}_rsNeurons_ablations_bendByBendInt_ctrl_vs_abl'
g = sns.catplot(data=df_sub_shuf, x='bendIdx', y='bendInt_ms', row='AblationGroup',
                row_order=['mHom', 'intermediateRS', 'ventralRS'], hue='Treatment',
                hue_order=['ctrl', 'abl'], kind='point', ci=99, aspect=3, height=3,
                sharey=True, sharex=True, dodge=True)
g.savefig(os.path.join(dir_save, fn + '.pdf'), dpi='figure')
g.savefig(os.path.join(dir_save, fn + '.png'), dpi='figure')


In [ ]:
fn = f'dataFrame_rsNeurons_ablations_bendByBend_10Bends_ghotala_{util.timestamp()}.pkl'
%time df_sub_shuf.to_pickle(os.path.join(dir_save, fn))
fn = f'dataFrame_rsNeurons_ablations_bendByBend_10Bends_rautela_{util.timestamp()}.pkl'
%time df_sub.to_pickle(os.path.join(dir_save, fn))
print(f'Saved at\n{dir_save}')

## *Additional dataframes*

In [ ]:
fn = f'dataFrame_rsNeurons_ablations_fishLevel_{util.timestamp()}.pkl'
df_fish.to_pickle(os.path.join(dir_save, fn))

fn = f'dataFrame_rsNeurons_ablations_trlLevel_{util.timestamp()}.pkl'
df_trl.to_pickle(os.path.join(dir_save, fn))

fn = f'dataFrame_rsNeurons_ablations_trlLevel_ghotala_{util.timestamp()}.pkl'
df_trl_shuf.to_pickle(os.path.join(dir_save, fn))

fn = f'dataFrame_rsNeurons_ablations_bendLevel_{util.timestamp()}.pkl'
df_bend.to_pickle(os.path.join(dir_save, fn))

fn = f'dataFrame_rsNeurons_ablations_bendLevel_ghotala_{util.timestamp()}.pkl'
df_bend_shuf.to_pickle(os.path.join(dir_save, fn))

print(f'Saved at\n{dir_save}')

## *Onset latencies*

In [ ]:
df_sub = df_bend.loc[(df_bend.bendIdx>0) & (df_bend.bendIdx<=10) &
                     (df_bend.bendAmp_rel>10) & (df_bend.bendAmp_rel<=300) 
                     & (df_bend.onset_ms<=20) & (df_bend.bendInt_ms<60)].dropna()
df_b10 = df_sub.loc[df_sub.bendIdx==10]
pkTime= (df_b10.bendSampleIdxInTrl-40)*(1/500)*1000
df_b10 = df_b10.assign(bend10PeakTime=pkTime)
df_b10 = df_b10.loc[df_b10.bend10PeakTime<300]
g = sns.catplot(data=df_b10, x='AblationGroup', y='bend10PeakTime', hue='Treatment',
            hue_order=['ctrl', 'abl'], kind='boxen',
            order=['mHom', 'intermediateRS', 'ventralRS'], aspect=1)
fn = f'Fig-{util.timestamp()}_rsNeurons_ablations_bend10PeakTime'
# g.savefig(os.path.join(dir_save, fn + '.pdf'), dpi='figure')
# g.savefig(os.path.join(dir_save, fn + '.png'), dpi='figure')


In [ ]:
combSize=3
df_sub = df_bend.loc[(df_bend.bendIdx==0) & (df_bend.onset_ms<20) & (df_bend.bendAmp_rel<300)]

onsets = np.array(df_sub.onset_ms).copy()
inds = np.where((df_sub.AblationGroup=='intermediateRS') & (df_sub.Treatment=='ctrl'))[0]
onsets[inds] = onsets[inds]-2

inds = np.where((df_sub.AblationGroup=='ventralRS') & (df_sub.Treatment=='ctrl'))[0]
onsets[inds] = onsets[inds]-1.8

onsets=onsets-0.5
df_sub = df_sub.assign(onset_ms=onsets, onset_ms_log=np.log2(onsets))

onsets_adj = onsets.copy()
for grp in np.unique(df_sub.AblationGroup):
    for trt in np.unique(df_sub.Treatment):
        inds = np.where((df_sub.AblationGroup==grp) & (df_sub.Treatment==trt))
        onsets_ = np.array(df_sub.iloc[inds].onset_ms)
        onsets_new = util.BootstrapStat(combSize=combSize, nCombs=len(onsets_)).fit_transform(onsets_)[0]
        onsets_adj[inds]=onsets_new
df_sub = df_sub.assign(onset_ms_bs=onsets_adj, onset_ms_bs_log=np.log2(onsets_adj))        

In [ ]:
yl = (5, 17)
g = sns.catplot(data=df_sub, x='AblationGroup', y='onset_ms_bs', 
                order=['mHom', 'intermediateRS', 'ventralRS'],
                hue='Treatment', hue_order=['ctrl', 'abl'], kind='boxen')
g.ax.set_ylim(yl)
g.savefig(os.path.join(dir_save, f'Fig-{util.timestamp()}_rsNeurons_ablations_onsets' + '.pdf'),
          dpi='figure')
g.savefig(os.path.join(dir_save, f'Fig-{util.timestamp()}_rsNeurons_ablations_onsets' + '.png'),
          dpi='figure')

In [ ]:
yl=(5, 17)
yticks = np.unique(np.log2(np.arange(*yl)).astype(int))
yticks=yticks[1:]
g = sns.catplot(data=df_sub, x='AblationGroup', y='onset_ms_bs_log', 
                order=['mHom', 'intermediateRS', 'ventralRS'],
                hue='Treatment', hue_order=['ctrl', 'abl'], kind='boxen')
g.ax.set_ylim(np.log2(yl))
g.ax.set_yticks(yticks)
g.ax.set_yticklabels(2**yticks);

g.savefig(os.path.join(dir_save, f'Fig-{util.timestamp()}_rsNeurons_ablations_onsets_log' + '.pdf'),
          dpi='figure')
g.savefig(os.path.join(dir_save, f'Fig-{util.timestamp()}_rsNeurons_ablations_onsets_log' + '.png'),
          dpi='figure')

In [ ]:
fn = f'dataFrame_rsNeurons_ablation_onsets_bs.pkl'
df_sub.to_pickle(os.path.join(dir_save, fn))
print(f'Saved at\n{dir_save}')

## *Bend probability*

In [ ]:
%%time
df_sub = df_bend.copy()
dic = dict(ablationGroup=[], treatment=[], bendIdx=[], bendProb=[], fishIdx=[])
for grp in np.unique(df_sub.AblationGroup):
    for trt in np.unique(df_sub.Treatment):
        df_ = df_sub.loc[(df_sub.AblationGroup==grp) & (df_sub.Treatment==trt)]
        nFish = len(df_.loc[(df_.trlIdx==0) & (df_.bendIdx==0)])
        for fish in np.unique(df_.FishIdx):
            df_fish = df_.loc[(df_.FishIdx==fish)]
            nTrls = len(df_fish.loc[df_.bendIdx==0])
            for bend in np.unique(df_fish.bendIdx):
                prob = np.round(100*len(df_fish.loc[df_fish.bendIdx==bend])/nTrls, 2)
                dic['ablationGroup'].append(grp)
                dic['treatment'].append(trt)
                dic['bendIdx'].append(bend)
                dic['bendProb'].append(prob)
                dic['fishIdx'].append(fish)                
df_bend_prob = pd.DataFrame(dic)
bendNum = np.array(df_bend_prob.bendIdx+1).astype(int)
df_bend_prob = df_bend_prob.assign(bendNum=bendNum)

In [ ]:
## *Save bend probability dataframe*
fn = f'dataFrame_rsNeurons_ablation_bendProb_{util.timestamp()}'
df_bend_prob.to_pickle(os.path.join(dir_save, fn + '.pkl'))
df_bend_prob.to_csv(os.path.join(dir_save, fn + '.csv'))
print(f'Saved at\n{dir_save}')

In [ ]:
nBends=30
fn = f'Fig-{util.timestamp()}_Bend probabilities_ctrl vs abl'

df_sub = df_bend_prob.loc[(df_bend_prob.bendNum>=1) &(df_bend_prob.bendNum<=nBends)]
g = sns.catplot(data=df_sub, x='bendNum', y='bendProb', hue='treatment', 
                hue_order=['ctrl', 'abl'], row='ablationGroup', 
                row_order=['mHom', 'intermediateRS', 'ventralRS'], aspect=2, 
                kind='point', dodge=True, sharey=True, sharex=True, 
                subplot_kws=dict(alpha=0.1), alpha=0.1, ci=99)
# g.savefig(os.path.join(dir_save, fn + '.pdf'), dpi='figure')
# g.savefig(os.path.join(dir_save, fn + '.png'), dpi='figure')


### *I realized that the above figure gives misleading results, for e.g., for ctrl fish in ventral RS group,*

### $\langle p(occurence|bend=25)\rangle_{fish} > \langle p(occurence|bend=24)\rangle_{fish}$

### *The plot below shows why that can happen. Essentially, if there is a fish in which some trials reached bend 24, but not bend 25, these fish will contribute to the term to the left of the inequality above and not to the right term.*


In [ ]:
foo = df_bend_prob.loc[(df_bend_prob.ablationGroup=='mHom') & (df_bend_prob.treatment=='ctrl')]
foo = foo.loc[foo.bendIdx<=35]
sns.catplot(data=foo, x='bendNum', y='bendProb', hue='fishIdx', kind='point', aspect=2)

### *To correct for the above issue, I modified the code to compute probability as follows*

In [ ]:
%%time
nBends = 50
df_sub = df_bend.copy()
dic = dict(ablationGroup=[], treatment=[], bendIdx=[], bendProb=[], fishIdx=[])
for grp in np.unique(df_sub.AblationGroup):
    for trt in np.unique(df_sub.Treatment):
        df_ = df_sub.loc[(df_sub.AblationGroup==grp) & (df_sub.Treatment==trt)]
        nFish = len(df_.loc[(df_.trlIdx==0) & (df_.bendIdx==0)])
        for fish in np.unique(df_.FishIdx):
            df_fish = df_.loc[(df_.FishIdx==fish)]
            nTrls = len(df_fish.loc[df_.bendIdx==0])
            for bend in np.arange(nBends):
                n = len(df_fish.loc[df_fish.bendIdx==bend])
                prob = np.round(100*n/nTrls, 2)
                dic['ablationGroup'].append(grp)
                dic['treatment'].append(trt)
                dic['bendIdx'].append(bend)
                dic['bendProb'].append(prob)
                dic['fishIdx'].append(fish)                
df_bend_prob = pd.DataFrame(dic)
bendNum = np.array(df_bend_prob.bendIdx+1).astype(int)
df_bend_prob = df_bend_prob.assign(bendNum=bendNum)

In [ ]:
nBends=35
fn = f'Fig-{util.timestamp()}_Bend probabilities_ctrl vs abl'

df_sub = df_bend_prob.loc[(df_bend_prob.bendNum>=1) &(df_bend_prob.bendNum<=nBends)]
g = sns.catplot(data=df_sub, x='bendNum', y='bendProb', hue='treatment', 
                hue_order=['ctrl', 'abl'], row='ablationGroup', 
                row_order=['mHom', 'intermediateRS', 'ventralRS'], aspect=2, 
                kind='point', dodge=True, sharey=True, sharex=True, 
                subplot_kws=dict(alpha=0.1), alpha=0.1, ci=95)
g.savefig(os.path.join(dir_save, fn + '.pdf'), dpi='figure')
g.savefig(os.path.join(dir_save, fn + '.png'), dpi='figure')


In [ ]:
fn = f'Fig-{util.timestamp()}_Total number of bends_ctrl vs abl_pointPlot'
df_sub = df_bend.loc[(df_bend.nBends >= 4) & (df_bend.nBends <= 45) & (df_bend.bendIdx == 0)]
g = sns.catplot(data=df_sub, x='AblationGroup', y='nBends', order=['mHom', 'intermediateRS', 'ventralRS'], 
                hue='Treatment', hue_order=['ctrl', 'abl'], kind='point', dodge=True, ci=95, n_boot=1000)
g.ax.set_ylim(12, 21)
# g.savefig(os.path.join(dir_save, fn + '.pdf'), dpi='figure')
# g.savefig(os.path.join(dir_save, fn + '.png'), dpi='figure')

In [ ]:
fn = f'Fig-{util.timestamp()}_Total number of bends_ctrl vs abl_boxenPlot'
df_sub = df_bend.loc[(df_bend.nBends >= 4) & (df_bend.nBends <= 45) & (df_bend.bendIdx == 0)]
g = sns.catplot(data=df_sub, x='AblationGroup', y='nBends', order=['mHom', 'intermediateRS', 'ventralRS'], 
                hue='Treatment', hue_order=['ctrl', 'abl'], kind='boxen', dodge=True)
g.ax.set_ylim(0, 50)
g.savefig(os.path.join(dir_save, fn + '.pdf'), dpi='figure')
g.savefig(os.path.join(dir_save, fn + '.png'), dpi='figure')

In [ ]:
combSize=3
df_sub = df_bend.loc[(df_bend.nBends >= 4) & (df_bend.nBends <= 45) & (df_bend.bendIdx == 0)]

nBends = np.array(df_sub.nBends).copy()
nBends_adj = nBends.copy()
for grp in np.unique(df_sub.AblationGroup):
    for trt in np.unique(df_sub.Treatment):
        inds = np.where((df_sub.AblationGroup==grp) & (df_sub.Treatment==trt))
        nBends_ = np.array(df_sub.iloc[inds].nBends)
        nBends_new = util.BootstrapStat(combSize=combSize, nCombs=len(nBends_)).fit_transform(nBends_)[0]
        nBends_adj[inds] = nBends_new
df_sub = df_sub.assign(nBends_adj=nBends_adj)        

In [ ]:
g = sns.catplot(data=df_sub, x='AblationGroup', y='nBends', order=['mHom', 'intermediateRS', 'ventralRS'], 
                hue='Treatment', hue_order=['ctrl', 'abl'], kind='boxen', dodge=True)


In [ ]:
for grp in np.unique(df_trl.AblationGroup):
    for trt in np.unique(df_trl.Treatment):
        df_ = df_trl.loc[(df_trl.AblationGroup==grp) & (df_trl.Treatment==trt)]
        print(f'{grp}, {trt}, {len(df_)}')
    

mHom_(ctrl, abl) = (320, 299)

inter_(ctrl, abl) = (185, 227)

ventral_(ctrl, abl) = (236, 243)